In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
bitcoin_data = pd.read_csv('BTC_Price_and_Indicators_Data.csv')

## Basic Moving Average Crossover Strategy
A Moving Average Crossover strategy is one of the simplest and most popular trading strategies. The moving averages help smooth out the price data to create a single flowing line, which makes it easier to identify the direction of the trend. When the short-term moving average (MA7) is above the long-term moving average (MA30), it may be a buy signal, and when it is below, it may be a sell signal.


In [3]:
# Generating Buy, Sell, and Hold signals
bitcoin_data['crossover_strategy'] = np.where(bitcoin_data['MA7'] > bitcoin_data['MA30'], 'BUY', 
                                              np.where(bitcoin_data['MA7'] < bitcoin_data['MA30'], 'SELL', 'HOLD'))

# Creating a Plotly figure
fig = go.Figure()

# Adding the closing price line
fig.add_trace(go.Scatter(x=bitcoin_data['Date'], y=bitcoin_data['Close'], name='Bitcoin Close Price', line=dict(width=2)))

# Adding the short-term moving average line
fig.add_trace(go.Scatter(x=bitcoin_data['Date'], y=bitcoin_data['MA7'], name='7 Day MA', line=dict(width=2)))

# Adding the long-term moving average line
fig.add_trace(go.Scatter(x=bitcoin_data['Date'], y=bitcoin_data['MA30'], name='30 Day MA', line=dict(width=2)))

# Adding Buy signal markers
fig.add_trace(go.Scatter(mode='markers', x=bitcoin_data[bitcoin_data['crossover_strategy'] == 'BUY']['Date'], 
                         y=bitcoin_data[bitcoin_data['crossover_strategy'] == 'BUY']['Close'],
                         marker=dict(color='Green', size=10, symbol='triangle-up'),
                         name='Buy Signal'))

# Adding Sell signal markers
fig.add_trace(go.Scatter(mode='markers', x=bitcoin_data[bitcoin_data['crossover_strategy'] == 'SELL']['Date'], 
                         y=bitcoin_data[bitcoin_data['crossover_strategy'] == 'SELL']['Close'],
                         marker=dict(color='Red', size=10, symbol='triangle-down'),
                         name='Sell Signal'))

# Updating layout and labels
fig.update_layout(title='Bitcoin Price and Moving Averages Crossover Strategy',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Displaying the figure
fig.show()


## RELATIVE STRENGTH INDEX (RSI) INDICATOR
The RSI is a popular technical analysis indicator used by many traders. It is used to analyse the momentum of a stock. It compares the recent gains with the recent losses. It gives an indication on the oversold or overbought condition of a stock.

**Strategy**
- Buy: When RSI is below 30.
- Sell: When RSI is above 70.
- Hold: When RSI is in between 30-70.

In [4]:
# Generating RSI-based trading signals
bitcoin_data['rsi_strategy'] = np.where(bitcoin_data['RSI'] < 30, 'BUY', 
                                        np.where(bitcoin_data['RSI'] > 70, 'SELL', 'HOLD'))

# Creating a Plotly figure
fig = go.Figure()

# Adding the closing price line
fig.add_trace(go.Scatter(x=bitcoin_data['Date'], y=bitcoin_data['Close'], name='Bitcoin Close Price', line=dict(width=2)))

# Adding Buy signal markers
fig.add_trace(go.Scatter(mode='markers', x=bitcoin_data[bitcoin_data['rsi_strategy'] == 'BUY']['Date'], 
                         y=bitcoin_data[bitcoin_data['rsi_strategy'] == 'BUY']['Close'],
                         marker=dict(color='Green', size=10, symbol='triangle-up'),
                         name='Buy Signal'))

# Adding Sell signal markers
fig.add_trace(go.Scatter(mode='markers', x=bitcoin_data[bitcoin_data['rsi_strategy'] == 'SELL']['Date'], 
                         y=bitcoin_data[bitcoin_data['rsi_strategy'] == 'SELL']['Close'],
                         marker=dict(color='Red', size=10, symbol='triangle-down'),
                         name='Sell Signal'))

# Updating layout and labels
fig.update_layout(title='Bitcoin Price and RSI Strategy',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Displaying the figure
fig.show()

## Moving Average Convergence Divergence (MACD) strategy
The MACD strategy is widely used for trading strategies as it can signal potential entry and exit points.

**Strategy**
- Buy: When MACD crosses above the Signal line.
- Sell: When MACD crosses below the Signal line.
- Hold: When no action is taken (i.e., no crossover occurs).

In [5]:
# Generating MACD-based trading signals
bitcoin_data['macd_strategy'] = 'HOLD'  # default to 'Hold'

bitcoin_data.loc[bitcoin_data['MACD'] > bitcoin_data['Signal_Line'], 'macd_strategy'] = 'BUY'
bitcoin_data.loc[bitcoin_data['MACD'] < bitcoin_data['Signal_Line'], 'macd_strategy'] = 'SELL'

bitcoin_data.loc[0, 'macd_strategy'] = 'HOLD' # Ensure the first position is not buying

# Adjust to only change positions on crossovers
bitcoin_data['macd_strategy'] = bitcoin_data['macd_strategy'].mask((bitcoin_data['macd_strategy'] != bitcoin_data['macd_strategy'].shift(1)), bitcoin_data['macd_strategy'])
bitcoin_data['macd_strategy'] = bitcoin_data['macd_strategy'].mask((bitcoin_data['macd_strategy'] == bitcoin_data['macd_strategy'].shift(1)), 'HOLD')

# Creating a Plotly figure
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, subplot_titles=('Bitcoin Price', 'MACD Analysis'))

# Adding the closing price line
fig.add_trace(go.Scatter(x=bitcoin_data['Date'], y=bitcoin_data['Close'], name='Bitcoin Close Price', line=dict(width=2)), row=1, col=1)

# Adding Buy signal markers
fig.add_trace(go.Scatter(mode='markers', x=bitcoin_data[bitcoin_data['macd_strategy'] == 'BUY']['Date'], 
                         y=bitcoin_data[bitcoin_data['macd_strategy'] == 'BUY']['Close'],
                         marker=dict(color='Green', size=10, symbol='triangle-up'),
                         name='Buy Signal'), row=1, col=1)

# Adding Sell signal markers
fig.add_trace(go.Scatter(mode='markers', x=bitcoin_data[bitcoin_data['macd_strategy'] == 'SELL']['Date'], 
                         y=bitcoin_data[bitcoin_data['macd_strategy'] == 'SELL']['Close'],
                         marker=dict(color='Red', size=10, symbol='triangle-down'),
                         name='Sell Signal'), row=1, col=1)

# Adding MACD line
fig.add_trace(go.Scatter(x=bitcoin_data['Date'], y=bitcoin_data['MACD'], name='MACD', line=dict(width=2)), row=2, col=1)

# Adding Signal Line
fig.add_trace(go.Scatter(x=bitcoin_data['Date'], y=bitcoin_data['Signal_Line'], name='Signal Line', line=dict(width=1, dash='dash')), row=2, col=1)

# Adding MACD Histogram
fig.add_trace(go.Bar(x=bitcoin_data['Date'], y=bitcoin_data['MACD_Histogram'], name='MACD Histogram'), row=2, col=1)

# Updating layout and labels
fig.update_layout(title='Bitcoin Price and MACD Strategy',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Displaying the figure
fig.show()

## Bolllinger Bands Strategy
The Bolllinger Bands strategy is widely used for trading strategies as it encompasses price levels diverging from a moving average (MA) by a certain number of standard deviations. 

**Strategy:**
- Buy Signal: When the price crosses below the lower Bollinger Band.
- Sell Signal: When the price crosses above the upper Bollinger Band.
- Hold: In all other situations.

In [6]:
# Initialize a column
bitcoin_data['bb_strategy'] = 'HOLD'  # default to 'Hold'

# Generate Buy & Sell signals
bitcoin_data.loc[bitcoin_data['Close'] < bitcoin_data['Bollinger_Low'], 'bb_strategy'] = 'BUY'
bitcoin_data.loc[bitcoin_data['Close'] > bitcoin_data['Bollinger_High'], 'bb_strategy'] = 'SELL'

# Ensure the first position is not buying
bitcoin_data.loc[0, 'bb_strategy'] = 'HOLD'

# Creating a Plotly figure
fig = go.Figure()

# Adding the closing price line
fig.add_trace(go.Scatter(x=bitcoin_data['Date'], y=bitcoin_data['Close'], name='Bitcoin Close Price', line=dict(width=2)))

# Adding Bollinger High line
fig.add_trace(go.Scatter(x=bitcoin_data['Date'], y=bitcoin_data['Bollinger_High'], name='Bollinger High', line=dict(width=1, dash='dash')))

# Adding Bollinger Low line
fig.add_trace(go.Scatter(x=bitcoin_data['Date'], y=bitcoin_data['Bollinger_Low'], name='Bollinger Low', line=dict(width=1, dash='dash')))


# Adding Buy signal markers
fig.add_trace(go.Scatter(mode='markers', x=bitcoin_data[bitcoin_data['bb_strategy'] == 'BUY']['Date'], 
                         y=bitcoin_data[bitcoin_data['bb_strategy'] == 'BUY']['Close'],
                         marker=dict(color='Green', size=10, symbol='triangle-up'),
                         name='Buy Signal'))

# Adding Sell signal markers
fig.add_trace(go.Scatter(mode='markers', x=bitcoin_data[bitcoin_data['bb_strategy'] == 'SELL']['Date'], 
                         y=bitcoin_data[bitcoin_data['bb_strategy'] == 'SELL']['Close'],
                         marker=dict(color='Red', size=10, symbol='triangle-down'),
                         name='Sell Signal'))

# Updating layout and labels
fig.update_layout(title='Bitcoin Price and Bollinger Band Strategy',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Displaying the figure
fig.show()

# Machine Learning Models

### 1. Split Data into Training and Testing Sets
Divide the data into training and testing sets. The training set will be used to train deep learning models, while the testing set will be used for evaluation.

In [7]:
from sklearn.model_selection import train_test_split

features = bitcoin_data[['MA7', 'MA30', 'RSI']]
target = bitcoin_data['Close']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42, shuffle=False)

### 2. Build Deep Learning Models
Select and construct deep learning models for Bitcoin price prediction. In this example, we'll use a recurrent neural network (RNN) with LSTM layers, a powerful choice for time-series data.

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Create LSTM model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)),
    Dense(1)
])

# Compile and fit the model
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Predictions
y_pred = model.predict(X_test)

Epoch 1/50
43/43 [==============================] - 3s 11ms/step - loss: 926636992.0000 - val_loss: 560781440.0000
Epoch 2/50
43/43 [==============================] - 0s 4ms/step - loss: 611059584.0000 - val_loss: 271219456.0000
Epoch 3/50
43/43 [==============================] - 0s 5ms/step - loss: 149017008.0000 - val_loss: 5201531.0000
Epoch 4/50
43/43 [==============================] - 0s 5ms/step - loss: 5011904.5000 - val_loss: 1104037.1250
Epoch 5/50
43/43 [==============================] - 0s 4ms/step - loss: 4393058.0000 - val_loss: 1123960.5000
Epoch 6/50
43/43 [==============================] - 0s 4ms/step - loss: 4375592.0000 - val_loss: 1103879.5000
Epoch 7/50
43/43 [==============================] - 0s 4ms/step - loss: 4377437.5000 - val_loss: 1089479.5000
Epoch 8/50
43/43 [==============================] - 0s 5ms/step - loss: 4377401.0000 - val_loss: 1179935.3750
Epoch 9/50
43/43 [==============================] - 0s 4ms/step - loss: 4397880.0000 - val_loss: 1126737.3750

### Step 4: Visualize the Results
Create compelling visualizations to showcase the model's predictions alongside actual Bitcoin prices using Matplotlib or other visualization librarie

In [9]:
fig = go.Figure()

# Actual close price
fig.add_trace(go.Scatter(x=bitcoin_data['Date'], y=bitcoin_data['Close'], mode='lines', name='Actual Prices', line=dict(color='blue')))

# Predicted close price
fig.add_trace(go.Scatter(x=bitcoin_data['Date'][y_test.index], y=y_pred.flatten(), mode='lines', name='Predicted Prices', line=dict(color='red', dash='dash')))

fig.update_layout(title='Actual vs. Predicted Bitcoin Prices for 2023-2024',
                   xaxis_title='Date',
                   yaxis_title='Bitcoin Price (USD)',
                   title_x=0.5,
                   hovermode='x unified')

fig.show()

In [10]:
predictions = model.predict(features).flatten()

# Initialize a new column for the strategy
bitcoin_data['ml_strategy'] = 'HOLD'

# Define a threshold for buying or selling
threshold = 0.03  # Example threshold, adjust according to your strategy

# Generate Buy & Sell signals
bitcoin_data.loc[predictions / bitcoin_data['Close'] - 1 > threshold, 'ml_strategy'] = 'BUY'
bitcoin_data.loc[predictions / bitcoin_data['Close'] - 1 < -threshold, 'ml_strategy'] = 'SELL'

# Create a Plotly graph object
fig = go.Figure()

# Add the closing price line
fig.add_trace(go.Scatter(x=bitcoin_data['Date'], y=bitcoin_data['Close'], name='Bitcoin Close Price', line=dict(width=2)))

# Add Buy signal markers
fig.add_trace(go.Scatter(mode='markers', x=bitcoin_data[bitcoin_data['ml_strategy'] == 'BUY']['Date'], 
                         y=bitcoin_data[bitcoin_data['ml_strategy'] == 'BUY']['Close'],
                         marker=dict(color='Green', size=10, symbol='triangle-up'),
                         name='Buy Signal'))

# Add Sell signal markers
fig.add_trace(go.Scatter(mode='markers', x=bitcoin_data[bitcoin_data['ml_strategy'] == 'SELL']['Date'], 
                         y=bitcoin_data[bitcoin_data['ml_strategy'] == 'SELL']['Close'],
                         marker=dict(color='Red', size=10, symbol='triangle-down'),
                         name='Sell Signal'))

# Update layout and labels
fig.update_layout(title='Bitcoin Price and ML Strategy',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  xaxis_rangeslider_visible=False)

# Display the figure
fig.show()

54/54 [==============================] - 0s 3ms/step
